In [38]:
from google.colab import files
uploaded = files.upload()


Saving NDVI_Karnataka.tif to NDVI_Karnataka.tif


In [41]:
import os
ndvi_path = "/content/NDVI_Karnataka.tif"
print("✅ File exists!" if os.path.exists(ndvi_path) else "❌ Upload failed.")


✅ File exists!


In [64]:
import rasterio
from pyproj import Transformer

# Karnataka district coordinates
district_coords = {
    "Bengaluru Urban": (12.9716, 77.5946),
    "Mandya": (12.5242, 76.8958),
    "Raichur": (16.2017, 77.3463),
    "Shivamogga": (13.9299, 75.5681),
    "Udupi": (13.3409, 74.7421)
}

ndvi_path = "/content/NDVI_Karnataka.tif"
ndvi_image = rasterio.open(ndvi_path)
transformer = Transformer.from_crs("EPSG:4326", ndvi_image.crs, always_xy=True)

location_data = {}

for district, (lat, lon) in district_coords.items():
    x, y = transformer.transform(lon, lat)
    row, col = ndvi_image.index(x, y)
    if 0 <= row < ndvi_image.height and 0 <= col < ndvi_image.width:
        ndvi_val = ndvi_image.read(1)[row, col] / 10000
    else:
        ndvi_val = 0.3
    location_data[district] = {
        "NDVI": round(ndvi_val, 3),
        "Temp": round(np.random.uniform(22, 28), 1),
        "Rainfall": int(np.random.uniform(80, 200))
    }

print(location_data)


{'Bengaluru Urban': {'NDVI': 0.3, 'Temp': 27.0, 'Rainfall': 197}, 'Mandya': {'NDVI': 0.3, 'Temp': 22.9, 'Rainfall': 167}, 'Raichur': {'NDVI': 0.3, 'Temp': 24.4, 'Rainfall': 188}, 'Shivamogga': {'NDVI': 0.3, 'Temp': 22.0, 'Rainfall': 165}, 'Udupi': {'NDVI': 0.3, 'Temp': 26.4, 'Rainfall': 121}}


In [65]:
import requests

OWM_API_KEY = "3960fa8490024e71a41c52dbc0db4cfe"

def fetch_weather(lat, lon):
    try:
        url = "https://api.openweathermap.org/data/2.5/weather"
        params = {"lat": lat, "lon": lon, "appid": OWM_API_KEY, "units": "metric"}
        response = requests.get(url, params=params).json()
        temp = response["main"]["temp"]
        rain = response.get("rain", {}).get("1h", 0)  # default 0 mm if not raining
        return round(temp, 1), int(rain)
    except:
        return round(np.random.uniform(22, 28), 1), int(np.random.uniform(80, 200))


In [66]:
for district, (lat, lon) in district_coords.items():
    x, y = transformer.transform(lon, lat)
    row, col = ndvi_image.index(x, y)
    if 0 <= row < ndvi_image.height and 0 <= col < ndvi_image.width:
        ndvi_val = ndvi_image.read(1)[row, col] / 10000
    else:
        ndvi_val = 0.3

    temp, rain = fetch_weather(lat, lon)

    location_data[district] = {
        "NDVI": round(ndvi_val, 3),
        "Temp": temp,
        "Rainfall": rain
    }

print(location_data)


{'Bengaluru Urban': {'NDVI': 0.3, 'Temp': 23.8, 'Rainfall': 0}, 'Mandya': {'NDVI': 0.3, 'Temp': 24.3, 'Rainfall': 0}, 'Raichur': {'NDVI': 0.3, 'Temp': 29.0, 'Rainfall': 0}, 'Shivamogga': {'NDVI': 0.3, 'Temp': 22.4, 'Rainfall': 0}, 'Udupi': {'NDVI': 0.3, 'Temp': 27.0, 'Rainfall': 0}}


In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
crops = ["Rice", "Wheat", "Coffee", "Maize", "Ragi"]
le.fit(crops)

X = []
y = []

for crop in crops:
    for _ in range(10):
        ndvi = round(np.random.uniform(0.3, 0.7), 3)
        temp = round(np.random.uniform(22, 30), 1)
        rain = int(np.random.uniform(80, 200))
        n = np.random.randint(50, 120)
        p = np.random.randint(30, 70)
        k = np.random.randint(20, 60)
        ph = round(np.random.uniform(5.5, 7.5), 1)
        crop_id = le.transform([crop])[0]
        yield_val = 3000 + crop_id * 200 + ndvi * 1000 + (temp - 22) * 30 + rain * 5
        X.append([crop_id, ndvi, temp, rain, n, p, k, ph])
        y.append(yield_val)

model = RandomForestRegressor()
model.fit(X, y)


RandomForestRegressor()

In [68]:
def predict_yield(location, crop, area_acres, n, p, k, ph):
    ndvi = location_data[location]["NDVI"]
    temp = location_data[location]["Temp"]
    rain = location_data[location]["Rainfall"]
    area_ha = area_acres * 0.4047
    crop_encoded = le.transform([crop])[0]
    features = [[crop_encoded, ndvi, temp, rain, n, p, k, ph]]
    y_pred = model.predict(features)[0]
    total = y_pred * area_ha
    return f" Predicted Yield: {int(y_pred)} kg/ha\n📦 Total for {area_acres} acres: {int(total)} kg"


In [76]:
# Improved Smart Farming Assistant with Voice, Hindi & Telugu Support, and NDVI Map
import gradio as gr
from transformers import pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
from PIL import Image
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import rasterio
from pyproj import Transformer
import matplotlib.pyplot as plt
import tempfile
import os

#  Translation pipeline
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

lang_codes = {
    "English": "eng_Latn",
    "Kannada": "kan_Knda",
    "Tamil": "tam_Taml",
    "Hindi": "hin_Deva",
    "Telugu": "tel_Telu"
}

def translate(text, src, tgt):
    try:
        src_lang = lang_codes[src]
        tgt_lang = lang_codes[tgt]
        result = translator(text, src_lang=src_lang, tgt_lang=tgt_lang)
        return result[0]['translation_text']
    except Exception as e:
        return f"⚠ Translation failed: {e}"

#  Load NDVI raster
ndvi_file = "/content/NDVI_Karnataka.tif"
ndvi_image = rasterio.open(ndvi_file)

#  District data
...  # Same district_coords and NDVI reading logic (trimmed here for brevity)

#  Visual NDVI Map
def generate_ndvi_map():
    ndvi = ndvi_image.read(1).astype(float)

    # Replace 0 or negative values with NaN for better visualization
    ndvi[ndvi <= 0] = np.nan

    # Normalize if values are >1 (assuming scaling factor)
    if np.nanmax(ndvi) > 1.0:
        ndvi /= 10000.0

    plt.figure(figsize=(8, 6))
    cmap = plt.cm.YlGn
    cmap.set_bad(color='lightgray')  # For NaNs

    plt.imshow(ndvi, cmap=cmap, vmin=0, vmax=1)
    plt.colorbar(label='NDVI (Normalized)')
    plt.title("🗺️ NDVI Map of Karnataka", fontsize=14)
    plt.axis("off")

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.tight_layout()
    plt.savefig(temp_file.name, bbox_inches='tight')
    plt.close()
    return temp_file.name


def detect_disease(image):
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model_disease(image_tensor)
        _, pred = torch.max(outputs, 1)
    return f"🩺 Detected: {class_names[pred.item()]}"


custom_css = """
body {
    background: linear-gradient(to right, #f0fff0, #e0ffff);
    font-family: 'Segoe UI', Tahoma, sans-serif;
    color: #2f4f4f;
}

h1, h2, h3 {
    color: #2e8b57;
    font-weight: bold;
}

.gr-button {
    background-color: #2e8b57 !important;
    color: white !important;
    border: none !important;
    border-radius: 12px !important;
    padding: 10px 20px !important;
    font-weight: bold;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.15);
    transition: 0.3s ease-in-out;
}
.gr-button:hover {
    background-color: #276749 !important;
    transform: scale(1.03);
}

.gr-textbox, .gr-slider, .gr-dropdown, .gr-image {
    border-radius: 10px !important;
    box-shadow: 0 2px 6px rgba(0, 0, 0, 0.08) !important;
    background-color: #ffffff !important;
    padding: 8px;
    font-size: 16px;
}

.gr-tabitem {
    background-color: #ffffffcc !important;
    border-radius: 12px;
    padding: 20px;
    margin: 10px;
}

.gr-image {
    border: 1px solid #d9f5d1 !important;
}

#ndvi-map-section {
    border: 2px solid #2e8b57;
    padding: 10px;
    border-radius: 12px;
    background-color: #f4fff4;
}
"""

#  Gradio UI with Audio + Map
with gr.Blocks(theme=gr.themes.Soft(), css=custom_css) as app:

    gr.Markdown("""
    #  Smart Farming Assistant
    Bringing together AI, Satellite Insights, and Multilingual Support with 🎤 Voice Commands
    """)

    with gr.Tab("📈 Yield Prediction"):
        with gr.Row():
            location = gr.Dropdown(list(location_data.keys()), label="📍 District")
            crop = gr.Dropdown(["Rice", "Wheat", "Coffee", "Maize", "Ragi"], label="🌾 Crop")
        with gr.Row():
            area = gr.Number(label="Land Area (acres)", value=1.0)
            ph = gr.Slider(3.5, 8.5, value=6.5, label="Soil pH")
        with gr.Row():
            n = gr.Slider(0, 150, value=90, label="N")
            p = gr.Slider(0, 100, value=45, label="P")
            k = gr.Slider(0, 100, value=30, label="K")
        yield_btn = gr.Button("🔍 Predict Yield")
        yield_output = gr.Textbox(label=" Prediction")
        yield_btn.click(predict_yield, inputs=[location, crop, area, n, p, k, ph], outputs=[yield_output])

    with gr.Tab(" AI Copilot"):
        question = gr.Textbox(lines=2, label="Ask your question")
        lang = gr.Dropdown(list(lang_codes.keys()), value="English", label=" Language")
        mic = gr.Audio(type="filepath", label="🎤 Voice Input", interactive=True)
        chat_btn = gr.Button("Ask")
        chat_output = gr.Textbox(label=" Copilot Answer")

        def process_voice(audio, text, language):
            import whisper
            model = whisper.load_model("base")
            result = model.transcribe(audio)
            return chatbot(result['text'], language)

        chat_btn.click(fn=chatbot, inputs=[question, lang], outputs=[chat_output])
        mic.change(fn=process_voice, inputs=[mic, question, lang], outputs=[chat_output])

    with gr.Tab("🩺 Plant Disease Detection"):
        img_input = gr.Image(type="pil", label="Upload Leaf Image")
        detect_btn = gr.Button("Detect Disease")
        detect_output = gr.Textbox(label=" Disease Type")
        detect_btn.click(detect_disease, inputs=[img_input], outputs=[detect_output])

    with gr.Tab("🗺️ NDVI Map"):
        map_btn = gr.Button("📍 Show NDVI Map")
        map_output = gr.Image(label="NDVI Karnataka")
        map_btn.click(fn=generate_ndvi_map, inputs=[], outputs=[map_output])

app.launch()



Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c6447156fa8299e0a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
